In [15]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.3.0


In [3]:
df = pd.read_csv('data/cruise_ship_info.csv')

In [4]:
X = df[['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']]
X_small = df[['Age', 'Tonnage', 'passengers', 'length', 'cabins']]
y = df['crew']
X.head()

,Age,Tonnage,passengers,length,cabins,passenger_density
0,6,30.277,6.94,5.94,3.55,42.64
1,6,30.277,6.94,5.94,3.55,42.64
2,26,47.262,14.86,7.22,7.43,31.80
3,11,110.000,29.74,9.53,14.88,36.99
4,17,101.353,26.42,8.92,13.21,38.36


In [5]:
train_features, test_features, train_target, test_target = train_test_split(X, y, random_state=42)

In [6]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))


In [16]:
lin_model = keras.Sequential([
    normalizer,
    layers.Dense(1)
])

lin_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')

logdir = 'tf_logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [17]:
%%time
history = lin_model.fit(
    train_features, train_target, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2,
    callbacks=[tensorboard_callback])


Instructions for updating:
use `tf.profiler.experimental.stop` instead.
CPU times: user 2.24 s, sys: 140 ms, total: 2.38 s
Wall time: 2.24 s


In [19]:
% tensorboard --logdir tf_logs/

UsageError: Line magic function `%` not found.
